In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import random

from tensorflow.keras import layers, models
from tensorflow.keras.layers import Layer, Dense, Conv1D, Multiply, Concatenate, Activation
from tensorflow.keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, Reshape
from tensorflow.keras.losses import CategoricalCrossentropy, BinaryCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy, BinaryAccuracy, AUC
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

from sklearn.metrics import f1_score, classification_report, average_precision_score, precision_recall_curve

In [2]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

In [4]:
# Squeeze-and-Excitation Attention
def se_block(x, r):

    s = layers.GlobalAveragePooling1D()(x)
    s = layers.Dense(x.shape[-1] // r, activation="relu")(s)
    s = layers.Dense(x.shape[-1], activation="sigmoid")(s)
    s = layers.Reshape((1, x.shape[-1]))(s)

    return layers.Multiply()([x, s])

In [5]:
# CNN Model with Attention for ECG classification
def build_ecg_model(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)

    #conv1
    x1 = layers.Conv1D(32, kernel_size=7, padding='same', use_bias=False, kernel_regularizer=l2(1e-4))(inputs)
    x1 = layers.BatchNormalization()(x1)
    x1 = layers.ReLU()(x1)
    x1 = se_block(x1, r=4)
    x1 = layers.MaxPooling1D(pool_size=2, strides=2)(x1)

    #conv2
    x2 = layers.Conv1D(64, kernel_size=5, padding='same', use_bias=False, kernel_regularizer=l2(1e-4))(x1)
    x2 = layers.BatchNormalization()(x2)
    x2 = layers.ReLU()(x2)
    x2 = se_block(x2, r=4)
    x2 = layers.MaxPooling1D(2)(x2)

    #conv3
    x3 = layers.Conv1D(128, kernel_size=3, padding='same', use_bias=False, kernel_regularizer=l2(1e-4))(x2)
    x3 = layers.BatchNormalization()(x3)
    x3 = layers.ReLU()(x3)
    #x3 = se_block(x3, r=4)
    x3 = layers.MaxPooling1D(pool_size=2, strides=2)(x3)

    #conv4
    x4 = layers.Conv1D(256, kernel_size=3, padding='same', use_bias=False, kernel_regularizer=l2(1e-4))(x3)
    x4 = layers.BatchNormalization()(x4)
    x4 = layers.ReLU()(x4)
    x4 = se_block(x4, r=4)
    x4 = layers.MaxPooling1D(pool_size=2, strides=2)(x4)

    # Global average pooling
    x = layers.GlobalAveragePooling1D()(x4)

    # Fully connected layers
    x = layers.Dense(32, activation='relu', kernel_regularizer=l2(1e-5))(x)
    x = layers.Dense(16, activation='relu', kernel_regularizer=l2(1e-5))(x)
    x = layers.Dense(8, activation='relu', kernel_regularizer=l2(1e-5))(x)
    x = layers.Dropout(0.2)(x)  # Dropout for regularization

    outputs = layers.Dense(num_classes, activation='sigmoid')(x)

    model = models.Model(inputs, outputs)

    return model

In [6]:
X_train = np.load('/content/drive/MyDrive/Colab Notebooks/X_train_normalized.npy')
X_val = np.load('/content/drive/MyDrive/Colab Notebooks/X_val_normalized.npy')
X_test = np.load('/content/drive/MyDrive/Colab Notebooks/X_test_normalized.npy')

# Load the labels
y_train = np.load('/content/drive/MyDrive/Colab Notebooks/y_train.npy', allow_pickle=True)
y_val = np.load('/content/drive/MyDrive/Colab Notebooks/y_val.npy', allow_pickle=True)
y_test = np.load('/content/drive/MyDrive/Colab Notebooks/y_test.npy', allow_pickle=True)

print("y_train shape:", y_train.shape)
print("y_val shape:", y_val.shape)
print("y_test shape:", y_test.shape)

print(y_train)

y_train shape: (17418, 5)
y_val shape: (2183, 5)
y_test shape: (2198, 5)
[[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 ...
 [1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]]


In [7]:
def class_distribution(y, name):
    counts = y.sum(axis=0)
    total = y.shape[0]
    print(f"\n{name} distribution:")
    for i, c in enumerate(counts):
        print(f"Class {i}: {int(c)} samples ({c/total:.2%})")

class_distribution(y_train, "Train")
class_distribution(y_val, "Validation")
class_distribution(y_test, "Test")


Train distribution:
Class 0: 7596 samples (43.61%)
Class 1: 4379 samples (25.14%)
Class 2: 4186 samples (24.03%)
Class 3: 3907 samples (22.43%)
Class 4: 2119 samples (12.17%)

Validation distribution:
Class 0: 955 samples (43.75%)
Class 1: 540 samples (24.74%)
Class 2: 528 samples (24.19%)
Class 3: 495 samples (22.68%)
Class 4: 268 samples (12.28%)

Test distribution:
Class 0: 963 samples (43.81%)
Class 1: 550 samples (25.02%)
Class 2: 521 samples (23.70%)
Class 3: 496 samples (22.57%)
Class 4: 262 samples (11.92%)


In [8]:
input_shape = (X_train.shape[1], X_train.shape[2])  # (sequence_length, num_leads)
num_classes = 5  # 5 diagnostic superclasses

set_seed(42)

model = build_ecg_model(input_shape, num_classes)

In [9]:
model.compile(optimizer=Adam(0.0001), loss = BinaryCrossentropy(from_logits=False), metrics=[BinaryAccuracy(name="bin_acc", threshold=0.5),
        AUC(name="auc_roc", curve="ROC", multi_label=True, num_labels=num_classes),
        AUC(name="auc_pr", curve="PR", multi_label=True, num_labels=num_classes)])

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 1000, 12)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 1000, 32)  │      2,688 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 1000, 32)  │        128 │ conv1d[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu (ReLU)        │ (None, 1000, 32)  │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 32)        │          0 │ re_lu[0][0]       │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 8)         │        264 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 32)        │        288 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 1, 32)     │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply (Multiply) │ (None, 1000, 32)  │          0 │ re_lu[0][0],      │
│                     │                   │            │ reshape[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d       │ (None, 500, 32)   │          0 │ multiply[0][0]    │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 500, 64)   │     10,240 │ max_pooling1d[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 500, 64)   │        256 │ conv1d_1[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_1 (ReLU)      │ (None, 500, 64)   │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 64)        │          0 │ re_lu_1[0][0]     │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 16)        │      1,040 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 64)        │      1,088 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_1 (Reshape) │ (None, 1, 64)     │          0 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_1          │ (None, 500, 64)   │          0 │ re_lu_1[0][0],    │
│ (Multiply)          │                   │            │ reshape_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_1     │ (None, 250, 64)   │          0 │ multiply_1[0][0]  │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 250, 128)  │     24,576 │ max_pooling1d_1[… │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 182,429 (712.61 KB)

 Trainable params: 181,469 (708.86 KB)

 Non-trainable params: 960 (3.75 KB)

In [ ]:
checkpoint = ModelCheckpoint('model_3.keras', monitor='val_auc_pr', save_best_only=True, mode='max', verbose=1)

earlystop = EarlyStopping(monitor="val_auc_pr", patience=15, restore_best_weights=True,  mode='max', verbose=1)

reduce_lr = ReduceLROnPlateau(monitor="val_auc_pr", mode="max", factor=0.5, patience=5, min_lr=1e-6, verbose=1)

history = model.fit(X_train,
                    y_train,
                    epochs=400,
                    batch_size=32,
                    validation_data=(X_val, y_val),
                    callbacks=[checkpoint, earlystop, reduce_lr])

Epoch 1/400
545/545 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - auc_pr: 0.3449 - auc_roc: 0.5954 - bin_acc: 0.6236 - loss: 0.6808
Epoch 1: val_auc_pr improved from -inf to 0.55989, saving model to model_3.keras
545/545 ━━━━━━━━━━━━━━━━━━━━ 25s 25ms/step - auc_pr: 0.3450 - auc_roc: 0.5954 - bin_acc: 0.6237 - loss: 0.6807 - val_auc_pr: 0.5599 - val_auc_roc: 0.7635 - val_bin_acc: 0.7554 - val_loss: 0.5188 - learning_rate: 1.0000e-04
Epoch 2/400
538/545 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - auc_pr: 0.5175 - auc_roc: 0.7477 - bin_acc: 0.7736 - loss: 0.5174
Epoch 2: val_auc_pr improved from 0.55989 to 0.64849, saving model to model_3.keras
545/545 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - auc_pr: 0.5177 - auc_roc: 0.7479 - bin_acc: 0.7738 - loss: 0.5172 - val_auc_pr: 0.6485 - val_auc_roc: 0.8331 - val_bin_acc: 0.8346 - val_loss: 0.4432 - learning_rate: 1.0000e-04
Epoch 3/400
541/545 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - auc_pr: 0.5855 - auc_roc: 0.8026 - bin_acc: 0.8095 - loss: 0.4609
Epoch 3: val_auc_pr improve

In [ ]:
results = model.evaluate(X_test, y_test, return_dict=True)
print(results)

69/69 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - auc_pr: 0.7581 - auc_roc: 0.8966 - bin_acc: 0.8783 - loss: 0.3249
{'auc_pr': 0.7670879364013672, 'auc_roc': 0.9009524583816528, 'bin_acc': 0.8751590847969055, 'loss': 0.32948434352874756}


In [ ]:
y_prob_val  = model.predict(X_val)
y_prob_test = model.predict(X_test)

thresholds = []
for j in range(y_val.shape[1]):
    precision, recall, th = precision_recall_curve(y_val[:, j], y_prob_val[:, j])
    f1 = 2*precision*recall / (precision + recall + 1e-12)
    best = np.argmax(f1)
    best_t = th[best] if best < len(th) else 0.5
    thresholds.append(best_t)

y_pred_test = (y_prob_test >= thresholds).astype(int)

print("Micro F1:", f1_score(y_test, y_pred_test, average="micro", zero_division=0))
print("Macro F1:", f1_score(y_test, y_pred_test, average="macro", zero_division=0))

# Per-class report
print(classification_report(y_test, y_pred_test, target_names=["NORM","MI","STTC","CD","HYP"], zero_division=0))

# Per-class PR-AUC (Average Precision)
ap = average_precision_score(y_test, y_prob_test, average=None)
for name, val in zip(["NORM","MI","STTC","CD","HYP"], ap):
    print(name, "AP:", val)

69/69 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step
Micro F1: 0.7520804438280166
Macro F1: 0.7137854052078898
              precision    recall  f1-score   support

        NORM       0.81      0.90      0.86       963
          MI       0.68      0.74      0.71       550
        STTC       0.69      0.78      0.73       521
          CD       0.73      0.68      0.70       496
         HYP       0.57      0.56      0.56       262

   micro avg       0.73      0.78      0.75      2792
   macro avg       0.70      0.73      0.71      2792
weighted avg       0.73      0.78      0.75      2792
 samples avg       0.74      0.78      0.74      2792

NORM AP: 0.916357110413718
MI AP: 0.7433182559835374
STTC AP: 0.7975330454313692
CD AP: 0.7871220492993554
HYP AP: 0.5954125175415994
